In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.simplefilter('ignore')

In [23]:
from google.colab import files
files = files.upload()

In [0]:
df_train = pd.read_csv('train.tsv', sep = '\t', thousands = ',')

In [0]:
df_test = pd.read_csv('test.tsv', sep = '\t', thousands = ',')

In [26]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 1000):
    review = re.sub('[^a-zA-Z]', ' ', df_train['Phrase'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

import string
string.punctuation    
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()

def own_analyser(phrase):
    phrase = phrase.split()
    for i in range(0,len(phrase)):
        k = phrase.pop(0)
        if k not in string.punctuation:
                phrase.append(lm.lemmatize(k).lower())    
    return phrase    

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = df_train['Phrase'] 
y = df_train.iloc[:, -1].values

In [0]:
from sklearn.model_selection import train_test_split
phrase_train,phrase_test,sentiment_train,sentiment_test = train_test_split(X,y,test_size=0.3)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

In [0]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('BOW',CountVectorizer(analyzer=own_analyser)),
                    ('tfidf',TfidfTransformer()),
                    ('classifier',MultinomialNB())])

In [32]:
pipeline.fit(phrase_train,sentiment_train)

Pipeline(memory=None,
         steps=[('BOW',
                 CountVectorizer(analyzer=<function own_analyser at 0x7fea51d43400>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [0]:
predictions = pipeline.predict(phrase_test)

In [34]:
predictions

array([2, 2, 2, ..., 2, 2, 3])

In [0]:
test_pred = pipeline.predict(df_test['Phrase'])

In [0]:
phrase_id = df_test['PhraseId'].values

In [0]:
Answer = pd.DataFrame({'PhraseId':phrase_id,'Sentiment':test_pred})

In [38]:
Answer

,PhraseId,Sentiment
0,156061,3
1,156062,3
2,156063,2
3,156064,3
4,156065,3
5,156066,3
6,156067,3
7,156068,3
8,156069,3
9,156070,2


In [39]:
filename = 'Sentiment_Analysis_1.csv'
Answer.to_csv(filename, index = False)
print('Saved file : ' + filename)

Saved file : Sentiment_Analysis_1.csv


In [0]:
from sklearn.metrics import accuracy_score

In [0]:
accuracy = accuracy_score(sentiment_test, predictions)

In [47]:
accuracy

0.5833867315989577

In [48]:
from sklearn.metrics import classification_report
print(classification_report(sentiment_test,predictions))

              precision    recall  f1-score   support

           0       0.74      0.03      0.06      2149
           1       0.51      0.25      0.33      8164
           2       0.60      0.90      0.72     23907
           3       0.52      0.38      0.44      9765
           4       0.61      0.04      0.08      2833

    accuracy                           0.58     46818
   macro avg       0.60      0.32      0.33     46818
weighted avg       0.58      0.58      0.53     46818

